In [1]:
import os
import sys
import time
import napari
import pickle
import h5py
import importlib
import numpy as np
import pandas as pd

from napari import Viewer

from napari.qt.threading import thread_worker
from matplotlib.backends.backend_qt5agg import FigureCanvas
from matplotlib.figure import Figure

from tifffile import imread,imsave,TiffFile
import matplotlib.pyplot as plt
import seaborn as sn

from magicgui import magicgui

sys.path.append(r'D:\imPy\libraries') 
sys.path.append(r'D:\BARC\napari_tracking_manual')

gallery_functions = importlib.import_module('gallery_functions')
my_napari = importlib.import_module('napari_display_functions')
fov_f = importlib.import_module('fovRingsLibrary')
gen = importlib.import_module('general_functions')

import warnings
warnings.filterwarnings('ignore')

In [181]:
importlib.reload(gallery_functions)
importlib.reload(my_napari)
importlib.reload(fov_f)
importlib.reload(gen)

<module 'general_functions' from 'D:\\BARC\\napari_tracking_manual\\general_functions.py'>

In [3]:
myFov = '01'

myDirIm = r'Z:\Wayne\20210618_RPE_p21_cycD1_DHB_H2B\tiffs'

# specify tracking channel
myFile_track_im = f'20210618_RPE_p21_cycD1_DHB_H2B_series_{myFov}_ch_01.tif'
track_intensity = 'intensity_01_nuc_corr' # name of the column

# specify additional channels
myFile_signal_im = [f'20210618_RPE_p21_cycD1_DHB_H2B_series_{myFov}_ch_02.tif',
                    f'20210618_RPE_p21_cycD1_DHB_H2B_series_{myFov}_ch_03.tif',
                   f'20210618_RPE_p21_cycD1_DHB_H2B_series_{myFov}_ch_04.tif']

colors_list = ['red','green','magenta']
names_list = ['DHB','cyclinD','p21']

# specify columns to plot
to_plot_signals = ['DHB_ratio','intensity_03_nuc_corr','intensity_04_nuc_corr','cyc_D_over_p21'] # name of the column
to_plot_names = ['DHB_ratio [cyt/nuc]','cyclinD','p21','Cyclin D/p21 (mean)']
to_plot_colors = ['red','green','blue','black']


myDirTracks = r'Z:\Wayne\20210618_RPE_p21_cycD1_DHB_H2B\tracking'
myDirTracks = os.path.join(myDirTracks,myFov)
dfDir = r'Z:\Wayne\20210618_RPE_p21_cycD1_DHB_H2B\tracking'
dfDir = os.path.join(dfDir,f'{myFov}_napari')

## Read in the tracking channel

In [5]:
%%time

# read in image
myIm = imread(os.path.join(myDirIm,myFile_track_im))

Wall time: 50.5 s


## Read in signal channels

In [6]:
%%time

myIm_signal_list = []
# read in additional signal images
for myFile in myFile_signal_im:
    temp = imread(os.path.join(myDirIm,myFile))
    myIm_signal_list.append(temp)

Wall time: 2min 32s


## Read in tracks in a form of labels

In [182]:
%%time

# read in labels as tracks
myLabels =[]

fovFiles = [x for x in os.listdir(myDirTracks) if f'series_{myFov}_' in x]

for myFile in fovFiles:
    
    # read a mask
    labels = imread(os.path.join(myDirTracks,myFile))
    
    myLabels.append(labels)
    
myLabels = np.array(myLabels)

Wall time: 1min 3s


## Get the full data frame

In [183]:
cellDataAll=pd.read_pickle(os.path.join(dfDir,f'cellPose_btrack_regionprops_bck_{myFov}.pkl'))
cellDataAll.drop(['file','x','y'],axis=1,inplace=True)

# sort 
cellDataAll = cellDataAll.sort_values(by=['track_id','t'])

In [184]:
# create an array for visualization of accepted points
selData=cellDataAll.loc[cellDataAll.accepted==True,:]
acceptedPoints = np.array([selData['t'],selData['centroid-0'],selData['centroid-1']]).T 

In [185]:
# generate data for the tracking layer
data,properties,graph = gen.trackData_from_df(cellDataAll)

## Main viewer

In [186]:
viewer = napari.Viewer()

# tracking layers
viewer.add_image(myIm,colormap='gray',contrast_limits=(0, 2000),opacity = 1)

track_layer=viewer.add_tracks(data, properties=properties, graph=graph,name='tracking')
track_layer.display_id=True

layer_inProgress = viewer.add_labels(myLabels,name='objects',opacity = 0.5)

# signal layers
for myIm_signal,myColor,myName in zip(myIm_signal_list,colors_list,names_list):
    viewer.add_image(myIm_signal,colormap=myColor,contrast_limits=(0, 4000),opacity = 1,visible=False,name = myName)

# helper layers
layer_accepted = viewer.add_points(acceptedPoints,name='accepted objects',face_color='green', opacity =0.5, ndim=3)
layer_mod = viewer.add_points([],name='modPoints',face_color='red',ndim=3)

## Add functions for modifying tracks

In [188]:
@magicgui(call_button='Cut Track')
def show_stack(viewer: Viewer):
    
    global cellDataAll
    global myLabels
    
    viewer,cellDataAll = my_napari.cut_track(viewer,cellDataAll)
    myLabels = viewer.layers['objects'].data
    
viewer.window.add_dock_widget(show_stack,area='right')


@magicgui(call_button='Merge Track')
def show_stack(viewer: Viewer):
    
    global cellDataAll
    global myLabels
    
    viewer,cellDataAll = my_napari.merge_track(viewer,cellDataAll)
    myLabels = viewer.layers['objects'].data
    
viewer.window.add_dock_widget(show_stack,area='right')


@magicgui(call_button='Connect Track')
def show_stack(viewer: Viewer):
    
    global cellDataAll
    global myLabels
    
    viewer,cellDataAll = my_napari.connect_track(viewer,cellDataAll)
    myLabels = viewer.layers['objects'].data
    
viewer.window.add_dock_widget(show_stack,area='right')

## Add functions for modifying labels

In [187]:
@magicgui(call_button='Modify label')
def show_stack(viewer: Viewer):
    
    global cellDataAll

    viewer,cellDataAll = my_napari.update_single_object(viewer,cellDataAll,myIm,myIm_signal_list)
    
viewer.window.add_dock_widget(show_stack,area='right')


@viewer.bind_key('u',overwrite=True)
def update_inProgress(viewer):
    
    global cellDataAll
    
    viewer,cellDataAll = my_napari.update_single_object(viewer,cellDataAll,myIm,myIm_signal_list)

## Mark track as accepted

In [189]:
# right click for accepting points
# toggles status of a track and visualizes it in the accepted objects layer if changed into accepted

@layer_inProgress.mouse_drag_callbacks.append
def accept_track(layer, event):

    if(event.button == 2):
        
        # look up cursor position
        x = int(viewer.cursor.position[1])
        y = int(viewer.cursor.position[2])
        
        # get data
        inProgressTracks = layer_inProgress.data
        
        # check which cell was clicked
        myTrackNum = inProgressTracks[viewer.dims.current_step[0],x,y]
        
        if myTrackNum > 0:

            # check status
            trackStatus = list(cellDataAll.loc[cellDataAll.track_id==myTrackNum,'accepted'])[0]

            # change status of this track
            cellDataAll.loc[cellDataAll.track_id==myTrackNum,'accepted'] = not(trackStatus)

            # regenerate accepted points
            selData=cellDataAll.loc[cellDataAll.accepted==True,:]
            acceptedPoints = np.array([selData['t'],selData['centroid-0'],selData['centroid-1']]).T 

            # update viewer    
            layer_accepted.data = acceptedPoints

## Stack viewer

In [190]:
def update_stack(viewer_stack,myTrack):
    
    global viewer
    
    data = viewer.layers['tracking'].data
    myLabels = viewer.layers['objects'].data
        
    # ask for an update
    stack_track,stack_labels,stack_signal_list = gallery_functions.stack_create_all(myIm,myLabels,myIm_signal_list,data,myTrack,imSize=100)
    
    # display new layers
    my_napari.display_set(viewer_stack,stack_track,stack_labels,stack_signal_list,colors_list,names_list,label_contour=0)
    
def update_graph(viewer_stack,myTrack):
    
    # remove previous graph
    h = viewer_stack.window._dock_widgets['']
    viewer_stack.window.remove_dock_widget(h)
        
    # add new graph
    mpl_widget = my_napari.create_graph_widget(track_intensity,to_plot_signals,to_plot_colors,to_plot_names,cellDataAll,myTrack) 
    h = viewer_stack.window.add_dock_widget(mpl_widget)

@magicgui(call_button='Show Stack')
def show_stack(viewer: Viewer):
    
    # find current track
    myLabel = viewer.layers['objects'].selected_label
    
    viewer_stack = napari.Viewer()
    update_stack(viewer_stack,myLabel)
    
    # select the right label
    viewer_stack.layers['objects'].selected_label = myLabel
    
    # init graph - there must be a more elegant solution (without this global handle)
    mpl_widget = my_napari.create_graph_widget(track_intensity,to_plot_signals,to_plot_colors,to_plot_names,cellDataAll,myLabel) 
    viewer_stack.window.add_dock_widget(mpl_widget)
    
    # add an update button
    # in the future - connect it to directly changing the selected layer
    @magicgui(call_button='Update Stack')
    def button_stack(viewer_stack: Viewer):

        myTrack = viewer_stack.layers['objects'].selected_label

        # update stack
        update_stack(viewer_stack,myTrack)

        # update graph
        update_graph(viewer_stack,myTrack)

    viewer_stack.window.add_dock_widget(button_stack,area='bottom')
    
viewer.window.add_dock_widget(show_stack,area='bottom')

## Gallery viewer

In [191]:
def update_gallery(viewer_gallery,myTrack):
    
    global viewer
    
    data = viewer.layers['tracking'].data
    myLabels = viewer.layers['objects'].data
        
    # ask for an update
    gallery_track,gallery_labels,gallery_signal_list = gallery_functions.gallery_create_all(myIm,myLabels,myIm_signal_list,data,myTrack,imSize=100)
    
    # display new layers
    my_napari.display_set(viewer_gallery,gallery_track,gallery_labels,gallery_signal_list,colors_list,names_list,label_contour=2)
    
def update_graph(viewer_gallery,myTrack):
    
    # remove previous graph
    h = viewer_gallery.window._dock_widgets['']
    viewer_gallery.window.remove_dock_widget(h)
        
    # add new graph
    mpl_widget = my_napari.create_graph_widget(track_intensity,to_plot_signals,to_plot_colors,to_plot_names,cellDataAll,myTrack) 
    h = viewer_gallery.window.add_dock_widget(mpl_widget)

@magicgui(call_button='Show Gallery')
def show_gallery(viewer: Viewer):
    
    # find current track
    myLabel = viewer.layers['objects'].selected_label
    
    viewer_gallery = napari.Viewer()
    update_gallery(viewer_gallery,myLabel)
    
    # select the right label
    viewer_gallery.layers['objects'].selected_label = myLabel
    
    # init graph - there must be a more elegant solution (without this global handle)
    mpl_widget = my_napari.create_graph_widget(track_intensity,to_plot_signals,to_plot_colors,to_plot_names,cellDataAll,myLabel) 
    viewer_gallery.window.add_dock_widget(mpl_widget)
    
    # add an update button
    # in the future - connect it to directly changing the selected layer
    @magicgui(call_button='Update Gallery')
    def button_gallery(viewer_gallery: Viewer):

        myTrack = viewer_gallery.layers['objects'].selected_label

        # update gallery
        update_gallery(viewer_gallery,myTrack)

        # update graph
        update_graph(viewer_gallery,myTrack)

    viewer_gallery.window.add_dock_widget(button_gallery,area='bottom')
    
viewer.window.add_dock_widget(show_gallery,area='bottom')

In [169]:
# get the position in time
myT = viewer.dims.current_step[0]

# get my label
myLabel = viewer.layers['objects'].selected_label

# create intensity image
intIm = gen.create_intensityImage(myIm,myIm_signal_list,myT)

# create mask with only a selected object
single_label_im = gen.create_singleLabel(myLabels,myT,myLabel)

# characterize new nucleus
cellData = gen.characterize_newNucleus(single_label_im,intIm)

# create ring image
x = int(cellData['centroid-0'])
y = int(cellData['centroid-1'])
single_label_ring = gen.make_ringImage(single_label_im,x,y,imSize=200)

# measure properties of the ring
ringData = gen.characterize_newRing(single_label_ring,intIm)

In [170]:
# check which cell it is
myLabel = list(cellData['label'])[0]

# put nucleus and ring data together
cellData = pd.merge(cellData,ringData,how='inner',on='label',suffixes=('_nuc', '_ring'))

# modify names
myNames = list(cellData.columns)
myNames[2] = 'centroid-0'
myNames[3] = 'centroid-1'
cellData.columns = myNames

# add aditional info
cellData['t'] = myT
cellData['track_id'] = myLabel

In [171]:
# collect information about this label and this time point to calculate 
info_track = cellDataAll.loc[:,['track_id','parent','root','generation','accepted']].drop_duplicates()
info_frame = cellDataAll.loc[cellDataAll.t==myT,['t','background_01','background_02','background_03','background_04']].drop_duplicates()

# merge it to the data of this frame
cellData = cellData.merge(info_track,on='track_id',how='left')

In [174]:
cellData

,label,area,centroid-0,centroid-1,major_axis_length,minor_axis_length,orientation,bbox-0,bbox-1,bbox-2,...,mean_intensity-0_ring,mean_intensity-1_ring,mean_intensity-2_ring,mean_intensity-3_ring,t,track_id,parent,root,generation,accepted
0,3000,906,1538.00883,1741.014349,37.180221,31.085755,-0.451923,1521,1725,1557,...,635.34472,775.976708,527.409938,1531.358696,21,3000,NaN,NaN,NaN,NaN


In [177]:
cellData.loc[0,'parent']

nan

In [178]:
if (cellData.loc[0,'parent'] == cellData.loc[0,'parent']):
    pass
else:
    cellData.parent = cellData.track_id
    cellData.generation = 0
    cellData.root = cellData.track_id
cellData = cellData.merge(info_frame,on='t')

In [179]:
cellData

,label,area,centroid-0,centroid-1,major_axis_length,minor_axis_length,orientation,bbox-0,bbox-1,bbox-2,...,t,track_id,parent,root,generation,accepted,background_01,background_02,background_03,background_04
0,3000,906,1538.00883,1741.014349,37.180221,31.085755,-0.451923,1521,1725,1557,...,21,3000,3000,3000,0,NaN,623.773443,256.137182,487.35491,1504.802679


In [180]:
# calculate corrected signals
for ch in np.arange(1,5):

    cellData[f'intensity_{str(ch).zfill(2)}_nuc_corr'] = cellData[f'mean_intensity-{ch-1}_nuc'] - cellData[f'background_{str(ch).zfill(2)}']
    cellData[f'intensity_{str(ch).zfill(2)}_ring_corr'] = cellData[f'mean_intensity-{ch-1}_ring'] - cellData[f'background_{str(ch).zfill(2)}']

# swap in the general data frame
what_to_drop = ((cellDataAll.t==myT) & (cellDataAll.track_id==myLabel))
drop_overlaping_neighbours = ((cellDataAll.t==myT) & (abs(cellDataAll['centroid-0']-cellData['centroid-0'][0])<10) & (abs(cellDataAll['centroid-1']-cellData['centroid-1'][0])<10))
what_to_drop =(what_to_drop | drop_overlaping_neighbours)
cellDataAll.drop(cellDataAll[what_to_drop].index,axis=0,inplace=True)
cellDataAll = cellDataAll.append(cellData,ignore_index=True)

In [143]:
len(cellDataAll.t==myT)

67804

In [146]:
cellDataAll['centroid-0']

361    1586.737288
448    1589.373526
535    1596.503807
621    1601.100000
708    1604.510559
          ...     
162    1534.082090
236    2221.344902
207    2000.168639
212    2024.750623
216    2045.793103
Name: centroid-0, Length: 67804, dtype: float64

In [148]:
cellData

,label,area,centroid-0,centroid-1,major_axis_length,minor_axis_length,orientation,bbox-0,bbox-1,bbox-2,...,background_03,background_04,intensity_01_nuc_corr,intensity_01_ring_corr,intensity_02_nuc_corr,intensity_02_ring_corr,intensity_03_nuc_corr,intensity_03_ring_corr,intensity_04_nuc_corr,intensity_04_ring_corr


In [144]:
(abs(cellDataAll['centroid-0']-cellData['centroid-0'][0])<10)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [126]:
myT,myLabel

(8, 3000)

In [104]:
cellDataAll.loc[((cellDataAll.track_id==myLabel) & (cellDataAll.t==myT)),:]

,label,area,centroid-0,centroid-1,orientation,major_axis_length,minor_axis_length,bbox-0,bbox-1,bbox-2,...,intensity_02_nuc_corr,intensity_02_ring_corr,background_03,intensity_03_nuc_corr,intensity_03_ring_corr,background_04,intensity_04_nuc_corr,intensity_04_ring_corr,DHB_ratio,cyc_D_over_p21
67831,152.0,564.0,1838.558511,1067.154255,-0.595808,29.845705,24.19023,1825.0,1054.0,1853.0,...,1166.490553,321.620777,515.813471,304.223763,60.084834,1583.356536,41.602684,-23.710585,NaN,NaN


In [116]:
cellDataAll.loc[((cellDataAll.track_id==116) & (cellDataAll.t==myT)),:]

,label,area,centroid-0,centroid-1,orientation,major_axis_length,minor_axis_length,bbox-0,bbox-1,bbox-2,...,intensity_02_nuc_corr,intensity_02_ring_corr,background_03,intensity_03_nuc_corr,intensity_03_ring_corr,background_04,intensity_04_nuc_corr,intensity_04_ring_corr,DHB_ratio,cyc_D_over_p21


In [106]:
# put nucleus and ring data together
cellData = pd.merge(cellData,ringData,how='inner',on='label',suffixes=('_nuc', '_ring'))

# modify names
myNames = list(cellData.columns)
myNames[2] = 'centroid-0'
myNames[3] = 'centroid-1'
cellData.columns = myNames

# add aditional info
cellData['t'] = myT
cellData['track_id'] = myLabel

# collect information about this label and this time point to calculate 
info_track = cellDataAll.loc[:,['track_id','parent','root','generation','accepted']].drop_duplicates()
info_frame = cellDataAll.loc[cellDataAll.t==myT,['t','background_01','background_02','background_03','background_04']].drop_duplicates()

# merge it to the data of this frame
cellData = cellData.merge(info_track,on='track_id')
cellData = cellData.merge(info_frame,on='t')

# calculate corrected signals
for ch in np.arange(1,5):

    cellData[f'intensity_{str(ch).zfill(2)}_nuc_corr'] = cellData[f'mean_intensity-{ch-1}_nuc'] - cellData[f'background_{str(ch).zfill(2)}']
    cellData[f'intensity_{str(ch).zfill(2)}_ring_corr'] = cellData[f'mean_intensity-{ch-1}_ring'] - cellData[f'background_{str(ch).zfill(2)}']


In [107]:
cellData

,label,area,centroid-0,centroid-1,major_axis_length,minor_axis_length,orientation,bbox-0,bbox-1,bbox-2,...,background_03,background_04,intensity_01_nuc_corr,intensity_01_ring_corr,intensity_02_nuc_corr,intensity_02_ring_corr,intensity_03_nuc_corr,intensity_03_ring_corr,intensity_04_nuc_corr,intensity_04_ring_corr
0,152,564,1838.558511,1067.154255,29.845705,24.19023,-0.595808,1825,1054,1853,...,515.813471,1583.356536,52.688308,4.522352,1166.490553,321.620777,304.223763,60.084834,41.602684,-23.710585


In [108]:
what_to_drop = ((cellDataAll.t==myT) & (cellDataAll.track_id==myLabel))
np.sum(what_to_drop)

1

In [109]:
drop_overlaping_neighbours = ((cellDataAll.t==myT) & (abs(cellDataAll['centroid-0']-cellData['centroid-0'][0])<10) & (abs(cellDataAll['centroid-1']-cellData['centroid-1'][0])<10))
np.sum(drop_overlaping_neighbours)

2

In [112]:
what_to_drop = (what_to_drop | drop_overlaping_neighbours)

In [114]:
cellDataAll.loc[what_to_drop,'track_id']

17623    116.0
67831    152.0
Name: track_id, dtype: float64

In [115]:
cellDataAll.drop(cellDataAll[what_to_drop].index,axis=0,inplace=True)

In [80]:
cellDataAll['centroid-0']-cellData['centroid-0']

0       -276.624838
1               NaN
2               NaN
3               NaN
4               NaN
            ...    
67826           NaN
67827           NaN
67828           NaN
67829           NaN
67830           NaN
Name: centroid-0, Length: 67831, dtype: float64

In [82]:
cellData['centroid-0'][0]

1863.362126245847

In [55]:
cellDataAll.loc[drop_overlaping_neighbours,'track_id']

17618    116.0
Name: track_id, dtype: float64

In [56]:
np.sum(what_to_drop | drop_overlaping_neighbours)

2

In [117]:
# modify the data for the layer
data,properties,graph = gen.trackData_from_df(cellDataAll)

# change tracks layer
viewer.layers['tracking'].data = data
viewer.layers['tracking'].properties = properties
viewer.layers['tracking'].graph = graph

In [18]:
# saving
# it should go under the button with a separate thread

myDirSave = r'Z:\Wayne\20210618_RPE_p21_cycD1_DHB_H2B\tracking'
myDirSave = os.path.join(myDirSave,f'{myFov}_napari')

# read in labels as tracks
imsave(os.path.join(myDirSave,f'corrected_labels_{myFov}.tif'),myLabels,dtype=myIm.dtype,shape=myIm.shape)

cellDataAll.to_pickle(os.path.join(dfDir,f'cellPose_btrack_regionprops_bck_corr_{myFov}.pkl'))

In [132]:
# swapping tracks based on 4 points - in development

@viewer.bind_key('r',overwrite=True)
def swap_points(viewer):

    swapPoints = viewer.layers['swapPoints'].data.astype(int)

    # check that you get a good set of swapPoints
    lenTest = (len(swapPoints) == 4)

    if lenTest:

        framesToSwap = list(set(swapPoints[:,0]))
        framesToSwap.sort()
        frameTest = (framesToSwap[1] == framesToSwap[0]+1)


        # read in in-progress tracks
        inProgressTracks = viewer.layers['inProgressStack'].data

        # check tracks
        myTracks = []
        for myPoint in swapPoints:

            myTracks.append(inProgressTracks[tuple(myPoint)])

        trackTest = (len(set(myTracks)) == 2)

        if (lenTest and frameTest and trackTest):

            # get tracks numbers
            track_1 = list((set(myTracks)))[0] 
            track_2 = list((set(myTracks)))[1]

            frame_change = np.max(framesToSwap)


            inProgressTracks[frame_change:,:,:][inProgressTracks[frame_change:,:,:]==track_1]=4095
            inProgressTracks[frame_change:,:,:][inProgressTracks[frame_change:,:,:]==track_2]=track_1
            inProgressTracks[frame_change:,:,:][inProgressTracks[frame_change:,:,:]==4095]=track_2

            # change labels layer
            viewer.layers.pop('inProgressStack')
            viewer.add_labels(inProgressTracks,name='inProgressStack',opacity = 0.5)


            # change graph
            children_1 = set(data[((properties['parent']==track_1) & (properties['generation']>0)),0])

            for child in children_1:

                graph[int(child)] = track_2

            children_2 = set(data[((properties['parent']==track_2) & (properties['generation']>0)),0])

            for child in children_2:

                graph[int(child)] = track_1


            # change tracking data
            track1_change = ((data[:,0]==track_1) & (data[:,1]>=frame_change))
            track2_change = ((data[:,0]==track_2) & (data[:,1]>=frame_change))

            data[track2_change,0] = track_1
            data[track1_change,0] = track_2

            # change info about the parent 
            track1_change = ((properties['parent']==track_1) & (data[:,1]>=frame_change))
            track2_change = ((properties['parent']==track_2) & (data[:,1]>=frame_change))

            properties['parent'][track1_change] = track_2
            properties['parent'][track2_change] = track_1

            # change info about the root
            root1 = properties['root'][track1_change][0]
            root2 = properties['root'][track2_change][0]

            track1_change = ((properties['root']==root1) & (data[:,1]>=frame_change))
            track2_change = ((properties['root']==root2) & (data[:,1]>=frame_change))

            properties['root'][track1_change] = root2
            properties['root'][track2_change] = root1

            # change info about the generation
            gen1 = properties['generation'][(properties['t']==(frame_change-1)) & (data[:,0]==track_1)]
            gen2 = properties['generation'][(properties['t']==(frame_change-1)) & (data[:,0]==track_2)]

            properties['generation'][track1_change] = properties['generation'][track1_change] + gen2 - gen1
            properties['generation'][track2_change] = properties['generation'][track2_change] + gen1 - gen2



            # update tracking data
            viewer.layers['data'].data = data
            viewer.layers['data'].properties = properties
            viewer.layers['data'].graph = graph

            # clean swap points
            viewer.layers['swapPoints'].data = []

            viewer.status='Tracks have been swapped.'

        else:

            viewer.status='Swap points are incorrect.'
    else:

        viewer.status='Swap points are incorrect.'

C:\ProgramData\Anaconda3\envs\cellpose\lib\site-packages\napari\layers\tracks\tracks.py:391: UserWarning: Previous color_by key 't' not present in new properties. Falling back to track_id
  UserWarning,


In [ ]:
# fix update of the small stack to nothing (at the moment it keeps the old cell)

In [ ]:
# move graphs from matplotlib to qt and add them vertical lines
# add numbering of the frames to gallery

In [ ]:
# what to do with ghost objects

In [ ]:
# key binding for linking function

# check if parent have different offspring

# no other offspring
# connect points

# other offspring
# connect
# add to the graph

# what if objects didn;t

In [ ]:
# label image should probably become a float image